![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV

filename_features = "features.txt"
filename_labels = "activity_labels.txt"

filename_subtrain = "./train/subject_train.txt"
filename_xtrain = "./train/X_train.txt"
filename_ytrain = "./train/y_train.txt"

filename_subtest = "./test/subject_test.txt"
ffilename_xtest = "./test/X_test.txt"
filename_ytest = "./test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])



C:\Users\lucia\AppData\Local\Temp\ipykernel_15852\1085637560.py:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])
C:\Users\lucia\AppData\Local\Temp\ipykernel_15852\1085637560.py:23: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
C:\Users\lucia\AppData\Local\Temp\ipykernel_15852\1085637560.py:27: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features['nome_var'].tolist())


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [2]:
%%time
clf = DecisionTreeClassifier(random_state=222, ccp_alpha=0.001).fit(X_train, y_train)

train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

print(f'Acurácia na base de treino:    {train_score*100:.1f}')
print(f'Acurácia na base de teste:    {test_score*100:.1f}')

Acurácia na base de treino:    97.6
Acurácia na base de teste:    88.1
CPU times: total: 6.05 s
Wall time: 6.35 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [4]:
%%time
prcomp = PCA(n_components=1).fit(X_train)

pc_train = prcomp.transform(X_train)
pc_test  = prcomp.transform(X_test)

clf2 = DecisionTreeClassifier(random_state=444, ccp_alpha=0.001).fit(pc_train, y_train)

train_score2 = clf2.score(pc_train, y_train)
test_score2 = clf2.score(pc_test, y_test)

print(f'Acurácia na base de treino:    {train_score2*100:.1f}')
print(f'Acurácia na base de teste:    {test_score2*100:.1f}')

Acurácia na base de treino:    50.0
Acurácia na base de teste:    45.7
CPU times: total: 188 ms
Wall time: 289 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [7]:
%%time
import time
component_numbers = [1, 2, 5, 10, 50]

train_accuracies = []
test_accuracies = []
execution_times = []

# Loop para fazer o processo em cada número de componentes:
for n_components in component_numbers:
    start_time = time.time()  # Start time
    
    # PCA
    prcomp = PCA(n_components=n_components).fit(X_train)
    pc_train = prcomp.transform(X_train)
    pc_test = prcomp.transform(X_test)

    # Ajuste da árvore
    clf = DecisionTreeClassifier(random_state=444, ccp_alpha=0.001)
    clf.fit(pc_train, y_train)

    # Acurácia
    train_score = clf.score(pc_train, y_train)
    test_score = clf.score(pc_test, y_test)

    train_accuracies.append(train_score)
    test_accuracies.append(test_score)
    
    end_time = time.time()  # End time
    execution_time = end_time - start_time  
    execution_times.append(execution_time)

#Armazenando os resultados num df para melhor visualização:
results_df = pd.DataFrame({
    'Component Number': component_numbers,
    'Train Accuracy': train_accuracies,
    'Test Accuracy': test_accuracies,
    'Execution Time (s)': execution_times
})
print(results_df)


   Component Number  Train Accuracy  Test Accuracy  Execution Time (s)
0                 1        0.499728       0.457075            0.267390
1                 2        0.612758       0.584662            0.300578
2                 5        0.846028       0.788599            0.288810
3                10        0.892274       0.820495            0.421035
4                50        0.916213       0.829657            1.291434
CPU times: total: 3.52 s
Wall time: 2.6 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

Tanto a acurácia quanto o tempo de processamento aumentaram com número de componentes. 